In [ ]:
#Importing Tensorflow and checking the versions
import tensorflow as tf
print(tf.__version__)

2.14.0


In [ ]:
'''Installing Kaggle to directly download the dataset and connect my google colab
directly to the Kaggle datasets'''
!pip install kaggle

In [ ]:
!pwd

/content


In [ ]:
#Importing the Kaggle.json file from the GDrive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!mkdir -p  ~/.kaggle
!cp '/content/drive/MyDrive/kaggle.json' ~/.kaggle/
#Permission change settings
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#Downloading the dataset
!kaggle competitions download -c digit-recognizer

 59% 9.00M/15.3M [00:00<00:00, 77.5MB/s]
100% 15.3M/15.3M [00:00<00:00, 104MB/s] 


In [ ]:
#Unizipping the files from the dataset
!unzip 'digit-recognizer.zip'

Archive:  digit-recognizer.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
from subprocess import check_output
print(check_output(["ls","../content"]).decode("utf8"))

digit-recognizer.zip
drive
sample_data
sample_submission.csv
test.csv
train.csv



In [ ]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
print(train.shape)
print(test.shape)

(42000, 785)
(28000, 784)


In [ ]:
train_y=train['label'].astype('float32')
train_x=train.drop(['label'],axis=1).astype('int32')
test_x=test.astype('float32')

train_y.shape,train_x.shape,test_x.shape


((42000,), (42000, 784), (28000, 784))

In [ ]:
train_x=train_x.values.reshape(-1,28,28,1)
train_x=train_x/255.0
test_x=test_x.values.reshape(-1,28,28,1)
test_x=test_x/255.0

train_x.shape,test_x.shape

((42000, 28, 28, 1), (28000, 28, 28, 1))

In [ ]:
#Onehot encoding
train_y = tf.keras.utils.to_categorical(train_y,10)
train_y.shape

(42000, 10)

In [ ]:
print(train_y[0:5,:])

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape=(28,28,1)),
  tf.keras.layers.Conv2D(32,(3,3),activation = 'relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'Same'),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'Same'),
  tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'Same'),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'Same'),
  tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  #tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                   

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('accuracy') > 0.999):
                print("\nReached 99.9% accuracy so cancelling training!")
                self.model.stop_training = True

callbacks = myCallback()

In [ ]:
Optimizer = tf.keras.optimizers.Adam(
            learning_rate=0.0005,
            beta_1=0.9,
            beta_2=0.999,
            epsilon=1e-07,
            name='Adam'
)
model.compile(optimizer=Optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, batch_size = 50, epochs = 20, callbacks=[callbacks])

Epoch 1/20
840/840 [==============================] - 155s 182ms/step - loss: 0.3407 - accuracy: 0.8892
Epoch 2/20
840/840 [==============================] - 154s 183ms/step - loss: 0.0830 - accuracy: 0.9752
Epoch 3/20
840/840 [==============================] - 152s 181ms/step - loss: 0.0604 - accuracy: 0.9826
Epoch 4/20
840/840 [==============================] - 147s 175ms/step - loss: 0.0492 - accuracy: 0.9854
Epoch 5/20
840/840 [==============================] - 153s 182ms/step - loss: 0.0410 - accuracy: 0.9883
Epoch 6/20
840/840 [==============================] - 155s 184ms/step - loss: 0.0379 - accuracy: 0.9888
Epoch 7/20
840/840 [==============================] - 154s 183ms/step - loss: 0.0334 - accuracy: 0.9900
Epoch 8/20
840/840 [==============================] - 148s 176ms/step - loss: 0.0270 - accuracy: 0.9917
Epoch 9/20
840/840 [==============================] - 148s 177ms/step - loss: 0.0250 - accuracy: 0.9923
Epoch 10/20
840/840 [==============================] - 150s 178m

In [ ]:
results = model.predict(test_x)

# select the index with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

875/875 [==============================] - 34s 38ms/step


In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("submission.csv",index=False)

In [ ]:
!kaggle competitions submit -c digit-recognizer -f submission.csv -m "Message"

100% 208k/208k [00:00<00:00, 627kB/s]  
Successfully submitted to Digit Recognizer